In [4]:
from pre_processing import df_all, df_week_player,df_week_team
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import numpy as np
import re
import os
import datetime
import altair as alt

In [5]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.max_rows = 4000
pd.options.display.max_seq_items = 2000

In [6]:
df_all["Date"].min()

Timestamp('2021-07-01 00:00:00')

In [7]:

metrics_classes = {
    "Intensity": ['Load Per Minute', 'Distance Per Minute'],
    "Agility": ['Acc 2m/s2 Total Effort','Acc 3m/s2 Total Effort', 'Dec 2m/s2 Total Effort', 'Dec 3m/s2 Total Effort'],
    "IMA": ['IMA COD(left)', 'IMA COD(right)'],
    "Volumn": ['Duration', 'Total Distance(m)', 'Total Player Load']
}

In [8]:
df_week_player.head()

,Player,Position,Team Name,Year,Week,Year-Week,Duration,Total Distance(m),Total Player Load,Acc 2m/s2 Total Effort,Acc 3m/s2 Total Effort,Dec 2m/s2 Total Effort,Dec 3m/s2 Total Effort,High Intensity Distance(m),Sprint Distance(m),Maximum Velocity(m/s),IMA COD(left),IMA COD(right),Load Per Minute,Distance Per Minute,IMA COD Imbalance,Is IMA Imbalance,Duration EWMA ACWR,Total Distance(m) EWMA ACWR,Total Player Load EWMA ACWR,Acc 2m/s2 Total Effort EWMA ACWR,Acc 3m/s2 Total Effort EWMA ACWR,Dec 2m/s2 Total Effort EWMA ACWR,Dec 3m/s2 Total Effort EWMA ACWR,High Intensity Distance(m) EWMA ACWR,Sprint Distance(m) EWMA ACWR,Maximum Velocity(m/s) EWMA ACWR,IMA COD(left) EWMA ACWR,IMA COD(right) EWMA ACWR,Load Per Minute EWMA ACWR,Distance Per Minute EWMA ACWR,is_Duration_abnormal,is_Total Distance(m)_abnormal,is_Total Player Load_abnormal,is_Acc 2m/s2 Total Effort_abnormal,is_Acc 3m/s2 Total Effort_abnormal,is_Dec 2m/s2 Total Effort_abnormal,is_Dec 3m/s2 Total Effort_abnormal,is_High Intensity Distance(m)_abnormal,is_Sprint Distance(m)_abnormal,is_Maximum Velocity(m/s)_abnormal,is_IMA COD(left)_abnormal,is_IMA COD(right)_abnormal,is_Load Per Minute_abnormal,is_Distance Per Minute_abnormal,Intensity Risk Score,Agility Risk Score,IMA Risk Score,Volumn Risk Score,Date,Season
0,Player1,Central Midfielder,Team1,2021,26,2021-W26,102.0,5162.00,491.00,3.0,0.0,7.0,2.0,96.70,0.00,25.70,50.0,39.0,4.813725,50.61,0.28,True,1.00,1.00,1.00,1.00,NaN,1.00,1.00,1.00,NaN,1.00,1.00,1.00,1.00,1.00,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0,2021-06-28,None
1,Player1,Central Midfielder,Team1,2021,27,2021-W27,51.2,5077.60,480.48,16.0,3.8,14.6,3.2,231.94,41.08,27.10,15.8,19.0,9.384375,99.17,-0.17,True,0.73,0.99,0.99,1.44,1.60,1.24,1.16,1.28,1.60,1.02,0.55,0.72,1.22,1.22,Low,Moderate,Moderate,High,High,Moderate,Moderate,Moderate,High,Moderate,Low,Low,Moderate,Moderate,0,2,2,1,2021-07-05,2021/22
2,Player1,Central Midfielder,Team1,2021,28,2021-W28,55.4,4672.20,501.36,16.0,2.6,16.2,4.8,202.18,12.28,27.40,24.8,27.6,9.049819,84.34,-0.10,False,0.88,0.96,1.02,1.18,1.04,1.16,1.26,1.05,0.65,1.01,0.93,1.02,1.08,1.02,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0,2021-07-12,2021/22
3,Player1,Central Midfielder,Team1,2021,29,2021-W29,57.6,4666.34,523.60,18.6,3.6,16.0,6.0,219.10,20.46,28.24,15.8,16.0,9.090278,81.01,-0.01,False,0.95,0.98,1.03,1.17,1.20,1.07,1.25,1.07,1.04,1.02,0.73,0.73,1.04,0.99,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Low,Low,Moderate,Moderate,0,0,2,0,2021-07-19,2021/22
4,Player1,Central Midfielder,Team1,2021,30,2021-W30,65.6,5769.14,561.52,16.8,1.4,16.8,7.0,233.08,14.08,27.44,22.4,25.4,8.559756,87.94,-0.12,True,1.05,1.11,1.06,1.03,0.61,1.07,1.22,1.07,0.82,1.00,1.02,1.08,0.99,1.04,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,1,0,0,2021-07-26,2021/22


In [9]:
player1 = df_week_player[df_week_player["Player"]=="Player1"]
player1.head(20)

,Player,Position,Team Name,Year,Week,Year-Week,Duration,Total Distance(m),Total Player Load,Acc 2m/s2 Total Effort,Acc 3m/s2 Total Effort,Dec 2m/s2 Total Effort,Dec 3m/s2 Total Effort,High Intensity Distance(m),Sprint Distance(m),Maximum Velocity(m/s),IMA COD(left),IMA COD(right),Load Per Minute,Distance Per Minute,IMA COD Imbalance,Is IMA Imbalance,Duration EWMA ACWR,Total Distance(m) EWMA ACWR,Total Player Load EWMA ACWR,Acc 2m/s2 Total Effort EWMA ACWR,Acc 3m/s2 Total Effort EWMA ACWR,Dec 2m/s2 Total Effort EWMA ACWR,Dec 3m/s2 Total Effort EWMA ACWR,High Intensity Distance(m) EWMA ACWR,Sprint Distance(m) EWMA ACWR,Maximum Velocity(m/s) EWMA ACWR,IMA COD(left) EWMA ACWR,IMA COD(right) EWMA ACWR,Load Per Minute EWMA ACWR,Distance Per Minute EWMA ACWR,is_Duration_abnormal,is_Total Distance(m)_abnormal,is_Total Player Load_abnormal,is_Acc 2m/s2 Total Effort_abnormal,is_Acc 3m/s2 Total Effort_abnormal,is_Dec 2m/s2 Total Effort_abnormal,is_Dec 3m/s2 Total Effort_abnormal,is_High Intensity Distance(m)_abnormal,is_Sprint Distance(m)_abnormal,is_Maximum Velocity(m/s)_abnormal,is_IMA COD(left)_abnormal,is_IMA COD(right)_abnormal,is_Load Per Minute_abnormal,is_Distance Per Minute_abnormal,Intensity Risk Score,Agility Risk Score,IMA Risk Score,Volumn Risk Score,Date,Season
0,Player1,Central Midfielder,Team1,2021,26,2021-W26,102.000000,5162.000000,491.000000,3.000000,0.000000,7.000000,2.000000,96.700000,0.000000,25.700000,50.000000,39.000000,4.813725,50.61,0.28,True,1.00,1.00,1.00,1.00,NaN,1.00,1.00,1.00,NaN,1.00,1.00,1.00,1.00,1.00,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0,2021-06-28,None
1,Player1,Central Midfielder,Team1,2021,27,2021-W27,51.200000,5077.600000,480.480000,16.000000,3.800000,14.600000,3.200000,231.940000,41.080000,27.100000,15.800000,19.000000,9.384375,99.17,-0.17,True,0.73,0.99,0.99,1.44,1.60,1.24,1.16,1.28,1.60,1.02,0.55,0.72,1.22,1.22,Low,Moderate,Moderate,High,High,Moderate,Moderate,Moderate,High,Moderate,Low,Low,Moderate,Moderate,0,2,2,1,2021-07-05,2021/22
2,Player1,Central Midfielder,Team1,2021,28,2021-W28,55.400000,4672.200000,501.360000,16.000000,2.600000,16.200000,4.800000,202.180000,12.280000,27.400000,24.800000,27.600000,9.049819,84.34,-0.10,False,0.88,0.96,1.02,1.18,1.04,1.16,1.26,1.05,0.65,1.01,0.93,1.02,1.08,1.02,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0,2021-07-12,2021/22
3,Player1,Central Midfielder,Team1,2021,29,2021-W29,57.600000,4666.340000,523.600000,18.600000,3.600000,16.000000,6.000000,219.100000,20.460000,28.240000,15.800000,16.000000,9.090278,81.01,-0.01,False,0.95,0.98,1.03,1.17,1.20,1.07,1.25,1.07,1.04,1.02,0.73,0.73,1.04,0.99,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Low,Low,Moderate,Moderate,0,0,2,0,2021-07-19,2021/22
4,Player1,Central Midfielder,Team1,2021,30,2021-W30,65.600000,5769.140000,561.520000,16.800000,1.400000,16.800000,7.000000,233.080000,14.080000,27.440000,22.400000,25.400000,8.559756,87.94,-0.12,True,1.05,1.11,1.06,1.03,0.61,1.07,1.22,1.07,0.82,1.00,1.02,1.08,0.99,1.04,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,1,0,0,2021-07-26,2021/22
5,Player1,Central Midfielder,Team1,2021,31,2021-W31,54.500000,4107.150000,374.200000,13.500000,2.000000,13.000000,4.000000,81.450000,3.000000,27.250000,18.000000,21.500000,6.866055,75.36,-0.16,True,0.92,0.86,0.80,0.89,0.92,0.89,0.80,0.51,0.27,0.99,0.89,0.95,0.87,0.93,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Low,Low,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0,2021-08-02,2021/22
6,Player1,Central Midfielder,Team1,2021,32,2021-W32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0.92,0.86,0.80,0.89,0.92,0.89,0.80,0.51,0.27,0.99,0.89,0.95,0.87,0.93,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,M

In [17]:
not_pass_metrics = {}
for key, value in metrics_classes.items():
    not_pass = [metric for metric in value if any(player1.head(8)[f"is_{metric}_abnormal"]!="Moderate")]
    not_pass_metrics[key] = not_pass

In [22]:
if "test" in not_pass_metrics:
    print("ok")
else:
    print("not ok")

not ok


In [6]:
# List of metrics to plot
metrics = ['Duration', 'Total Distance(m)', 'Total Player Load', 'Acc 2m/s2 Total Effort',
           'Acc 3m/s2 Total Effort', 'Dec 2m/s2 Total Effort', 'Dec 3m/s2 Total Effort',
           'High Intensity Distance(m)', 'Sprint Distance(m)', 'Maximum Velocity(m/s)',
           'IMA COD(left)', 'IMA COD(right)']

metrics_classes = {
    "Intensity": ['Load Per Minute', 'Distance Per Minute'],
    "Agility": ['Acc 2m/s2 Total Effort','Acc 3m/s2 Total Effort', 'Dec 2m/s2 Total Effort', 'Dec 3m/s2 Total Effort'],
    "IMA": ['IMA COD(left)', 'IMA COD(right)'],
    "Volumn": ['Duration', 'Total Distance(m)', 'Total Player Load']
}


In [7]:
plot_df = df_week_player[df_week_player["Player"] == "Player1"].dropna()

In [8]:
ewma_acwr = alt.Chart(plot_df).mark_line(point=alt.OverlayMarkDef(color="#FF7676"), color="#FA8072").encode(
alt.X("Date"),
alt.Y(f'Total Distance(m) EWMA ACWR'),
alt.Tooltip(["Date:T", "Total Distance(m) EWMA ACWR", "Total Distance(m):Q"])
)

# Create the dashed horizontal line at y = 1.5
upper_ewma_acwr = alt.Chart(pd.DataFrame({'y': [1.5]})).mark_rule(strokeDash=[5, 5], color='#FF6969').encode(
    y='y:Q'
)

# Create the dashed horizontal line at y = 0.8
lower_ewma_acwr = alt.Chart(pd.DataFrame({'y': [0.8]})).mark_rule(strokeDash=[5, 5], color='#FF6969').encode(
    y='y:Q'
)
    
chart = alt.Chart(plot_df).mark_bar(color="#0F52BA").encode(
    alt.X("Date:T", axis=alt.Axis(title="Date")),
    alt.Y("Total Distance(m):Q", axis=alt.Axis(title="Total Distance(m)")),
    alt.Tooltip(["Date:T", "Total Distance(m):Q", "Total Distance(m) EWMA ACWR"])
)

ewma_acwr_combined = alt.layer(
    ewma_acwr,
    upper_ewma_acwr,
    lower_ewma_acwr
).resolve_scale(
    y='shared'  # Share y-axis for the EWMA ACWR related charts
)


combined_chart = alt.layer(
    chart,
    ewma_acwr_combined
).resolve_scale(
    y='independent'  # Allow independent y-axes
).properties(
    title="Total Distance and EWMA ACWR Over Time"
)

In [61]:
combined_chart

alt.LayerChart(...)

In [63]:
metrics_classes

{'Intensity': ['Load Per Minute', 'Distance Per Minute'],
 'Agility': ['Acc 2m/s2 Total Effort',
  'Acc 3m/s2 Total Effort',
  'Dec 2m/s2 Total Effort',
  'Dec 3m/s2 Total Effort'],
 'IMA': ['IMA COD(left)', 'IMA COD(right)'],
 'Volumn': ['Duration', 'Total Distance(m)', 'Total Player Load']}